# PxW

Testing our wrapper for the very first time. Can I implement this fast?

In [5]:
#import prefect
import PxW 

#from prefect import Task, Flow

In [7]:
# Check if the package gets installed correctly.
weka = PxW.Weka()

In [87]:
weka.fit()

## Fit

Fitting a decision tree with Weka works!

In [68]:
weka_cmd = """java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar"""

tree = "weka.classifiers.trees.J48"
tree_cmd = """{} -C 0.25 -M 2""".format(tree)

model_fn = "tree.model"
model_cmd = """-d {}""".format(model_fn)

data_fn = "../data/raw/iris.arff"
data_cmd = "-t {}".format(data_fn)

cmd = """{} {} {} -no-cv {}""".format(weka_cmd, tree_cmd, data_cmd, model_cmd)

shell = ShellTask()

with Flow("fit") as f:
    fit = shell(command=cmd)

status = f.run()

[2019-08-22 15:30:41,456] INFO - prefect.FlowRunner | Beginning Flow run for 'fit'
[2019-08-22 15:30:41,458] INFO - prefect.FlowRunner | Starting flow run.
[2019-08-22 15:30:41,464] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-08-22 15:30:41,466] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-08-22 15:30:41,467] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-08-22 15:30:41,802] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-08-22 15:30:41,803] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


In [88]:
print(status.result[fit].result.decode("utf-8"))


Options: -C 0.25 -M 2 

=== Classifier model (full training set) ===

J48 pruned tree
------------------

petal width (cm) <= 0.6: 0.0 (50.0)
petal width (cm) > 0.6
|   petal width (cm) <= 1.7
|   |   petal length (cm) <= 4.9: 1.0 (48.0/1.0)
|   |   petal length (cm) > 4.9
|   |   |   petal width (cm) <= 1.5: 2.0 (3.0)
|   |   |   petal width (cm) > 1.5: 1.0 (3.0/1.0)
|   petal width (cm) > 1.7: 2.0 (46.0/1.0)

Number of Leaves  : 	5

Size of the tree : 	9


Time taken to build model: 0.15 seconds

Time taken to test model on training data: 0 seconds

=== Error on training data ===

Correctly Classified Instances         147               98      %
Incorrectly Classified Instances         3                2      %
Kappa statistic                          0.97  
Mean absolute error                      0.0233
Root mean squared error                  0.108 
Relative absolute error                  5.2482 %
Root relative squared error             22.9089 %
Total Number of Instances      

## Predict

Next step is predict, but now that all kind of rolls well, this is probably trivial. Except some data-trickery afterwards

In [132]:
weka_cmd = """java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar"""

tree = "weka.classifiers.trees.J48"
tree_cmd = """{}""".format(tree)

model_fn = "tree.model"
model_cmd = """-l {}""".format(model_fn)

data_fn = "../data/raw/iris.arff"
data_cmd = "-T {}".format(data_fn)

out_fn = "pred.csv"
out_cmd = ""

cmd = """{} {} {} {} -p 0 """.format(weka_cmd, tree_cmd, data_cmd, model_cmd)

shell = ShellTask()


# Execution
with Flow("predict") as f:
    predict = shell(command=cmd)

status = f.run()

[2019-08-22 15:57:12,410] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-08-22 15:57:12,412] INFO - prefect.FlowRunner | Starting flow run.
[2019-08-22 15:57:12,416] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-08-22 15:57:12,417] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-08-22 15:57:12,418] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-08-22 15:57:12,601] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-08-22 15:57:12,602] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


In [134]:
s = status.result[predict].result.decode("utf-8")

In [138]:
print(s)



=== Predictions on test data ===

    inst#     actual  predicted error prediction
        1      1:0.0      1:0.0       1 
        2      1:0.0      1:0.0       1 
        3      1:0.0      1:0.0       1 
        4      1:0.0      1:0.0       1 
        5      1:0.0      1:0.0       1 
        6      1:0.0      1:0.0       1 
        7      1:0.0      1:0.0       1 
        8      1:0.0      1:0.0       1 
        9      1:0.0      1:0.0       1 
       10      1:0.0      1:0.0       1 
       11      1:0.0      1:0.0       1 
       12      1:0.0      1:0.0       1 
       13      1:0.0      1:0.0       1 
       14      1:0.0      1:0.0       1 
       15      1:0.0      1:0.0       1 
       16      1:0.0      1:0.0       1 
       17      1:0.0      1:0.0       1 
       18      1:0.0      1:0.0       1 
       19      1:0.0      1:0.0       1 
       20      1:0.0      1:0.0       1 
       21      1:0.0      1:0.0       1 
       22      1:0.0      1:0.0       1 
       23    

In [186]:
from io import StringIO

df = pd.read_csv(StringIO(s),
                 index_col=0,
                 header=1,
                 usecols=[0,2],
                 lineterminator='\n',
                 sep='\s+')

In [187]:
df

,predicted
inst#,
1,1:0.0
2,1:0.0
3,1:0.0
4,1:0.0
5,1:0.0
6,1:0.0
7,1:0.0
8,1:0.0
9,1:0.0


In [151]:
s[:200]

'\n\n=== Predictions on test data ===\n\n    inst#     actual  predicted error prediction\n        1      1:0.0      1:0.0       1 \n        2      1:0.0      1:0.0       1 \n        3      1:0.0      1:0.0  '

In [148]:
df

,inst# actual predicted error prediction
0,1 1:0.0 1:0.0 1
1,2 1:0.0 1:0.0 1
2,3 1:0.0 1:0.0 1
3,4 1:0.0 1:0.0 1
4,5 1:0.0 1:0.0 1
5,6 1:0.0 1:0.0 1
6,7 1:0.0 1:0.0 1
7,8 1:0.0 1:0.0 1
8,9 1:0.0 1:0.0 1
9,10 1:0.0 1:0.0 1


In [124]:
import pandas as pd

In [137]:
pd.DataFrame(s)

ValueError: DataFrame constructor not properly called!



=== Predictions on test data ===

    inst#     actual  predicted error prediction
        1      1:0.0      1:0.0       1 
        2      1:0.0      1:0.0       1 
        3      1:0.0      1:0.0       1 
        4      1:0.0      1:0.0       1 
        5      1:0.0      1:0.0       1 
        6      1:0.0      1:0.0       1 
        7      1:0.0      1:0.0       1 
        8      1:0.0      1:0.0       1 
        9      1:0.0      1:0.0       1 
       10      1:0.0      1:0.0       1 
       11      1:0.0      1:0.0       1 
       12      1:0.0      1:0.0       1 
       13      1:0.0      1:0.0       1 
       14      1:0.0      1:0.0       1 
       15      1:0.0      1:0.0       1 
       16      1:0.0      1:0.0       1 
       17      1:0.0      1:0.0       1 
       18      1:0.0      1:0.0       1 
       19      1:0.0      1:0.0       1 
       20      1:0.0      1:0.0       1 
       21      1:0.0      1:0.0       1 
       22      1:0.0      1:0.0       1 
       23    

In [129]:
print(status.result[predict].result.decode("utf-8"))



=== Predictions on test data ===

    inst#     actual  predicted error prediction
        1      1:0.0      1:0.0       1 
        2      1:0.0      1:0.0       1 
        3      1:0.0      1:0.0       1 
        4      1:0.0      1:0.0       1 
        5      1:0.0      1:0.0       1 
        6      1:0.0      1:0.0       1 
        7      1:0.0      1:0.0       1 
        8      1:0.0      1:0.0       1 
        9      1:0.0      1:0.0       1 
       10      1:0.0      1:0.0       1 
       11      1:0.0      1:0.0       1 
       12      1:0.0      1:0.0       1 
       13      1:0.0      1:0.0       1 
       14      1:0.0      1:0.0       1 
       15      1:0.0      1:0.0       1 
       16      1:0.0      1:0.0       1 
       17      1:0.0      1:0.0       1 
       18      1:0.0      1:0.0       1 
       19      1:0.0      1:0.0       1 
       20      1:0.0      1:0.0       1 
       21      1:0.0      1:0.0       1 
       22      1:0.0      1:0.0       1 
       23    

In [108]:
status.result[predict]

<Success: "Task run succeeded.">


=== Classifier model (full training set) ===

J48 pruned tree
------------------

petal width (cm) <= 0.6: 0.0 (50.0)
petal width (cm) > 0.6
|   petal width (cm) <= 1.7
|   |   petal length (cm) <= 4.9: 1.0 (48.0/1.0)
|   |   petal length (cm) > 4.9
|   |   |   petal width (cm) <= 1.5: 2.0 (3.0)
|   |   |   petal width (cm) > 1.5: 1.0 (3.0/1.0)
|   petal width (cm) > 1.7: 2.0 (46.0/1.0)

Number of Leaves  : 	5

Size of the tree : 	9


Time taken to build model: 0 seconds

Time taken to test model on test data: 0.01 seconds

=== Error on test data ===

Correctly Classified Instances         147               98      %
Incorrectly Classified Instances         3                2      %
Kappa statistic                          0.97  
Mean absolute error                      0.0233
Root mean squared error                  0.108 
Total Number of Instances              150     


=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      RO

In [53]:
from prefect import Flow
from prefect.tasks.shell import ShellTask

task = ShellTask()

with Flow("testrun") as f:
    weka_cmd = ""
    tree_cmd = "weka.classifiers.trees.J48 -C 0.25 -M 2"
    data_fnm = "../data/raw/iris.arff"
    
    output = """
    -classifications "weka.classifiers.evaluation.output.prediction.CSV -p first"
    """ '
    
    
    cmd = "{} {} -t {}".format(weka_cmd, tree_cmd, data_fnm)
    pwd = task(command=cmd)

state = f.run()

SyntaxError: EOL while scanning string literal (<ipython-input-53-208a854ed353>, line 13)

In [54]:
cmd

'java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.J48 -C 0.25 -M 2 -t ../data/raw/iris.arff'

In [52]:
state.result[pwd].result

b'\nOptions: -C 0.25 -M 2 \n\n=== Classifier model (full training set) ===\n\nJ48 pruned tree\n------------------\n\npetal width (cm) <= 0.6: 0.0 (50.0)\npetal width (cm) > 0.6\n|   petal width (cm) <= 1.7\n|   |   petal length (cm) <= 4.9: 1.0 (48.0/1.0)\n|   |   petal length (cm) > 4.9\n|   |   |   petal width (cm) <= 1.5: 2.0 (3.0)\n|   |   |   petal width (cm) > 1.5: 1.0 (3.0/1.0)\n|   petal width (cm) > 1.7: 2.0 (46.0/1.0)\n\nNumber of Leaves  : \t5\n\nSize of the tree : \t9\n\n\nTime taken to build model: 0.15 seconds\n\nTime taken to test model on training data: 0 seconds\n\n=== Error on training data ===\n\nCorrectly Classified Instances         147               98      %\nIncorrectly Classified Instances         3                2      %\nKappa statistic                          0.97  \nMean absolute error                      0.0233\nRoot mean squared error                  0.108 \nRelative absolute error                  5.2482 %\nRoot relative squared error             22.

In [30]:
from prefect import Flow
from prefect.tasks.shell import ShellTask
from prefect.utilities.debug import raise_on_exception

task = ShellTask(helper_script="java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar")

command = "weka.classifiers.trees.J48 -C 0.25 -M 2 -t iris.arff"

with Flow("My Flow") as f:
    # both tasks will be executed in home directory
    train_tree = task(command=command)
    
with raise_on_exception():
    out = f.run()

[2019-08-22 14:20:56,639] INFO - prefect.FlowRunner | Beginning Flow run for 'My Flow'
[2019-08-22 14:20:56,641] INFO - prefect.FlowRunner | Starting flow run.
[2019-08-22 14:20:56,644] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-08-22 14:20:56,645] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-08-22 14:20:56,645] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-08-22 14:20:56,719] ERROR - prefect.FlowRunner | Unexpected error while running flow: FAIL('Command failed with exit code 127: b\'Usage: java [-options] class [args...]\\n           (to execute a class)\\n   or  java [-options] -jar jarfile [args...]\\n           (to execute a jar file)\\nwhere options include:\\n    -d32\\t  use a 32-bit data model if available\\n    -d64\\t  use a 64-bit data model if available\\n    -server\\t  to select the "server" VM\\n    -zero\\t  to select the "zero" VM\\n    -dcevm

FAIL: Command failed with exit code 127: b'Usage: java [-options] class [args...]\n           (to execute a class)\n   or  java [-options] -jar jarfile [args...]\n           (to execute a jar file)\nwhere options include:\n    -d32\t  use a 32-bit data model if available\n    -d64\t  use a 64-bit data model if available\n    -server\t  to select the "server" VM\n    -zero\t  to select the "zero" VM\n    -dcevm\t  to select the "dcevm" VM\n                  The default VM is server,\n                  because you are running on a server-class machine.\n\n\n    -cp <class search path of directories and zip/jar files>\n    -classpath <class search path of directories and zip/jar files>\n                  A : separated list of directories, JAR archives,\n                  and ZIP archives to search for class files.\n    -D<name>=<value>\n                  set a system property\n    -verbose:[class|gc|jni]\n                  enable verbose output\n    -version      print product version and

In [19]:
out.message

'Some reference tasks failed.'

In [28]:
out.is_failed()

True